### Format

    Input
    """
    질문: <question>
    풀이: <reasoning>
    정답: <answer>

    질문: <question>
    풀이: 
    """
    
    
    Output
    """
    <reasoning>
    정답 : <answer>
    """
    

## 각각의 데이터는 같음
    - /userhomes/philhoon/kt-ai-challenge/result
    # kor-gsm8k-one_shot_cot_finetuning-test.jsonl
    # kor-gsm8k-peft2-test.jsonl

    # kor-gsm8k-one_shot_cot_finetuning-train.csv
    # kor-gsm8k-peft2-train.csv

In [1]:
import json
import requests
from src import utils
import time
import os
import csv
import re

In [2]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [ ]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api-peft1.json'
api = utils.read_json_file(api_path)

In [ ]:
BODY_TEMP = api['body']

In [ ]:
model = model_api(api['url'], api['headers'])

In [ ]:
# input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
input_file = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft1-test.jsonl'

In [ ]:
data = utils.read_jsonlines(input_file)

In [ ]:
len(data)

In [ ]:
# os.makedirs(output_path, exist_ok=True)
output_path = '/userhomes/philhoon/kt-ai-challenge/result'
output_file_name = 'peft1-result-ko-gsm8k-test.jsonl'
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')
# jsonl_writer = utils.JSONLWriter(output_file)

In [ ]:
# def get_gsm8k_zs(que):
#     prompt = f'질문 : {que} \n정답 : '
#     return prompt

In [ ]:
# def get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s):
#     prompt = f"질문: {kor_que}"
#     output = f"풀이: {kor_ans} 정답: {ans_s}"
#     return prompt, output

In [ ]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [ ]:
# from pprint import pprint

In [ ]:
def remove_line_slit(context):
    context = re.sub(r"\n", " ", context)
    return context

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [ ]:
from pprint import pprint

In [ ]:
cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    prompt = ins['kor_question_reason']

#     print(prompt)
    body = get_body(prompt, BODY_TEMP)
#     print(body)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('----')
    
#     break
    # checking whether there is an answer 
#     test = kor_que.split('\n')
    
#     if len(test[0]) != len(kor_que):
#         split_cnt += 1
#         continue
    
#     print(f'kor_que')
#     print(f'{kor_que}')
#     print(f'kor_ans')
#     print(f'{kor_ans}')
#     print(f'ans_s')
#     print(f'{ans_s}')
    
    ins['peft1_infer'] = res_context
#     jsonl_writer.write_json_line(ins)
    
    if cnt == 10:
        break
    else:
        cnt += 1

In [ ]:
len(data) - split_cnt

In [ ]:
len(input_lst)

In [ ]:
len(output_lst)

In [ ]:
def save_csv_file(file_path, sentence_a_list, sentence_b_list):
    with open(file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        for sentence_a, sentence_b in zip(sentence_a_list, sentence_b_list):
            writer.writerow([sentence_a, sentence_b])

In [ ]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [ ]:
output_file = output_path + '/kor-gsm8k-reason-train.csv'

In [ ]:
output_file

In [ ]:
save_csv_file(output_file, input_lst, output_lst)

In [ ]:
input_lst[1]

In [ ]:
output_lst[1]